In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
import portfolio_insurance as pi
import requests
import json
import time

# Data

In [2]:
dt = pd.read_csv('./../data/snp2.csv')
dt['yyyy'] = dt['Date'].str[:4]
price = dt[['Date', 'SnP', 'yyyy']]
ret = dt[['Date', 'Return', 'yyyy']]
rf = dt[['Date', 'RF', 'yyyy']]

price = price.set_index('Date')
ret = ret.set_index('Date')
rf = rf.set_index('Date')

vol = ret.rolling(window=12).std()
vol[:11] = vol[11:11+11]
vol['yyyy'] = ret['yyyy']


full_year = np.load('./../result/year/full_year.npy', allow_pickle=True).reshape(1)[0]

# Bootstrapping

In [3]:
win = 12
T = win/12
trad_filter = 0.00

In [9]:

for protection_level in [1.0]:
    for multiplier in [7]:
#     for multiplier in range(50,100,5):
        strategies = ['SP', 'CPPI95']
        a = { 'SP':0.95, 'CPPI95':protection_level}
        cost = { 'SP':False, 'CPPI95':False}
        func1 = { 'SP':pi.get_E }
        func2 = { 'SP':pi.get_cppi, 'CPPI95':pi.get_cppi }
        m = multiplier
        E, res = {}, {}
        res_final = {}

        for item in strategies:
            res_final[item] = []

        start_year = 1928
        end_year = start_year + 92
        window_size = 1

        # Generate the list of lists
        year_lists = [[str(year) for year in range(year_i, year_i + window_size)] for year_i in range(start_year, end_year - window_size + 1)]

        # Print the lists
        # year_lists[-1]

        final_w = {}
        for item in strategies:
            final_w[item] = {}
            for yyyy_list in list(np.arange(int(year_lists[0][0]),int(year_lists[-1][0]))):
                final_w[item][yyyy_list] = []




        for yyyy_list in year_lists[:-1]:
            print('from: ', yyyy_list[0])
            print('to: ', yyyy_list[-1])

            price_ = price[price['yyyy'].isin(yyyy_list)]['SnP']
            ret_ = ret[ret['yyyy'].isin(yyyy_list)]['Return']
            rf_ = rf[rf['yyyy'].isin(yyyy_list)]['RF']
            vol_ = vol[vol['yyyy'].isin(yyyy_list)]['Return']

        #     price_ = np.array(price_)
        #     vol_ = np.array(vol_)
            ret_ = np.array(ret_)
            rf_ = np.array(rf_)
            price_ = price_ / price_[0]
            rf_ = rf_ / 100


            for item in strategies:


                if item in ['SP']:
                    E[item] = func1[item](price_, ret_, vol_, T, a[item], rf_, trad_filter)
                    res[item] = func2[item](E['SP'][0], ret_, m, a[item], T, np.mean(rf_), cost[item])

                elif item in ['CPPI95']:
                    E[item] = func2[item](E['SP'][0], ret_, m, a[item], T, np.mean(rf_), cost[item])[1]
                    res[item] = func2[item](E['SP'][0], ret_, m, a[item], T, np.mean(rf_), cost[item])[0]


                res_final[item].append(E[item])
                final_w[item][int(yyyy_list[0])] = E[item]


        #     for item in strategies:

        #         plt.plot(price_[:300], label='BH')
        #         plt.plot(res['CPPI95'][:300], label='CPPI95')
        #         plt.legend()
        #         plt.show()
        #         plt.close()


        final_w_sequence = {'CPPI95':{}}
        years_to_include = 35

        for key in final_w['CPPI95']:
            # Initialize an empty list for each key
            final_w_sequence['CPPI95'][key] = []
            for i in range(years_to_include):
                # Check if the year exists in the final_w dictionary
                if key + i in final_w['CPPI95']:
                    # Extend the list for the key with the list for the current year
                    final_w_sequence['CPPI95'][key].extend(final_w['CPPI95'][key + i])


        w_dict = {}
        for item in full_year[35][0]:#[1928, 1929, 1930, 1931, 1932, 1936, 1937, 1938, 1945, 1946, 1947, 1961, 1962, 1963, 1967, 1968, 1969, 1970, 1972, 1973, 1974, 1975]:#list(final_w_sequence['SP95'].keys())[:48]:
            w_dict[item] = final_w_sequence['CPPI95'][item]

        pd.DataFrame(w_dict).to_csv('./../w/CPPI{}_{}.csv'.format(int(protection_level*100), multiplier))
        
        